# Create final dataset

Now we put all data together:
- Synthetic text data A1 to C2
- Administrative news bulletins
- Legal decisions from one of the [cantonal courts](https://www.baurekursgericht-zh.ch/). 
- Decisions of the Cantonal Council ([«Regierungsratsbeschlüsse»](https://www.zh.ch/de/politik-staat/gesetze-beschluesse/beschluesse-des-regierungsrates.html)).

**Imports**

In [2]:
import pandas as pd
from pandarallel import pandarallel
import warnings
import spacy

nlp = spacy.load("de_core_news_sm")

pd.options.mode.chained_assignment = None
pd.options.display.max_rows = 500
pd.options.display.max_seq_items = 500
pandarallel.initialize(progress_bar=False)
warnings.simplefilter("ignore", category=UserWarning)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


**Constants and functions**

In [3]:
INPUT_DIR = "_input/"

N_SAMPLES = 720

# Load and concat data

### Administrative news bulletins

In [15]:
admin = pd.read_parquet(f"{INPUT_DIR}zh_news.parq")
admin.drop(columns=["url"], inplace=True)
admin = admin.melt()
admin.rename(columns={"variable": "text_type", "value": "text"}, inplace=True)
admin.text_type = admin.text_type.map({"text": "Admin News"})
assert admin.text.duplicated().sum() == 0
admin["text_length"] = admin.text.apply(lambda x: len(x.split()))
admin = admin[admin.text_length > 100]
admin = admin[admin.text_length < 500]

admin = admin.sample(N_SAMPLES, random_state=42)
print(admin.shape)
display(admin.head())

(720, 3)


,text_type,text,text_length
560,Admin News,Mehrere unbekannte Personen haben am Sonntagab...,137
997,Admin News,Ein Radfahrer hat sich am frühen Montagnachmit...,131
262,Admin News,Bei einer Hausdurchsuchung hat die Kantonspoli...,254
381,Admin News,Die Kantonsarchäologie hat im Furttal erstmals...,412
925,Admin News,Bei einem Selbstunfall sind am Samstagnachmitt...,162


### Legal decisions Baurekursgericht

In [9]:
legal = pd.read_parquet(f"{INPUT_DIR}legal.parq")
display(legal.head())
legal.shape

,text_type,text,text_length
0,Legal,2613 jenseits der Strasse errichteten Gebäudes...,119
1,Legal,Wie sich aus den einschlägigen Bestimmungen de...,410
2,Legal,a-d PBG grundsätzlich nicht mehr möglich. Sie ...,350
3,Legal,"Die Rekurrentin ist Gebührenschuldnerin, aber ...",443
4,Legal,BRKE III Nr. 281 und 282/1991 vom 18. Dezember...,464


(720, 3)

### Decisions cantonal council (RRBs)

In [12]:
rrbs = pd.read_parquet(f"{INPUT_DIR}rrbs.parq")
display(rrbs.head())
rrbs.shape

,text,text_type
0,Auszug aus dem Protokoll des Regierungsrates d...,RRBs
1,Auszug aus dem Protokoll des Regierungsrates d...,RRBs
2,Auszug aus dem Protokoll des Regierungsrates d...,RRBs
3,Auszug aus dem Protokoll des Regierungsrates d...,RRBs
4,Auszug aus dem Protokoll des Regierungsrates d...,RRBs


(720, 2)

### Synthetic CEFR samples

In [13]:
cefr = pd.read_parquet(f"{INPUT_DIR}cefr_synthetic.parq")
cefr = cefr.melt(id_vars=["model"], value_vars=["A1", "A2", "B1", "B2", "C1", "C2"])
cefr.columns = ["model", "text_type", "text"]

cefr.reset_index(drop=True, inplace=True)
cefr.drop(columns=["model"], inplace=True)
print(cefr.shape)
cefr.head()

(4320, 2)


,text_type,text
0,A1,Ein Mann geht auf der Straße. Er sieht eine Ka...
1,A1,Max arbeitet im Büro. Er schreibt an einem Dok...
2,A1,Heute Morgen bin ich zur Arbeit gegangen. Ich ...
3,A1,Der Arbeiter hält einen Hammer. Er baut ein Ha...
4,A1,Zwei Freunde treffen sich im Park. Der eine fr...


In [17]:
df = pd.concat([admin, legal, rrbs, cefr])
df.reset_index(drop=True, inplace=True)
df = df[["text_type", "text"]]
df.to_parquet(f"{INPUT_DIR}/zix_dataset.parq")
df.text_type.value_counts()

text_type
Admin News    720
Legal         720
RRBs          720
A1            720
A2            720
B1            720
B2            720
C1            720
C2            720
Name: count, dtype: int64

# Inspect standard CEFR vocabulary

The [Goethe Institute](https://www.goethe.de) provides standard CEFR vocabularies for levels A1, A2 and B2.

The source PDFs are available here:
- [Level A1](https://www.goethe.de/pro/relaunch/prf/de/A1_SD1_Wortliste_02.pdf)
- [Level A2](https://www.goethe.de/pro/relaunch/prf/en/Goethe-Zertifikat_A2_Wortliste.pdf)
- and [Level B2](https://www.goethe.de/pro/relaunch/prf/en/Goethe-Zertifikat_B1_Wortliste.pdf).

We'll check how much of the words in a given text overlap with these vocabularies.

In [11]:
nlp = spacy.load("de_core_news_sm")


def lemmatize_text(text, swiss=False):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.is_alpha]
    tokens = [x.lower() for x in tokens]
    if swiss:
        tokens = [x.replace("ß", "ss") for x in tokens]
    return " ".join(tokens)

In [12]:
cefr = pd.read_parquet(f"zix/data/cefr_vocab.parq")
cefr.level.value_counts().sort_index()

level
A1     807
A2     607
B1    1803
Name: count, dtype: int64

In [13]:
# Simple text.
text = """Der Bund fragt Leute, bevor er neue Gesetze macht. Er will wissen, ob die Ideen gut sind. Der Bund fragt die Kantone, Parteien und Verbände. Er will von ihnen hören. Die Verbände sind von Gemeinden, Städten und Bergen. Auch von der Wirtschaft. Manchmal fragt der Bund auch andere Leute. Der Bund will sicherstellen, dass die Ideen richtig sind. Und dass sie umgesetzt werden können. Er will auch wissen, ob die Leute damit einverstanden sind."""

# Complex text.
text = """Als Vernehmlassungsverfahren wird diejenige Phase innerhalb des Vorverfahrens der Gesetzgebung bezeichnet, in der Vorhaben des Bundes von erheblicher politischer, finanzieller, wirtschaftlicher, ökologischer, sozialer oder kultureller Tragweite auf ihre sachliche Richtigkeit, Vollzugstauglichkeit und Akzeptanz hin geprüft werden. Die Vorlage wird zu diesem Zweck den Kantonen, den in der Bundesversammlung vertretenen Parteien, den Dachverbänden der Gemeinden, Städte und der Berggebiete, den Dachverbänden der Wirtschaft sowie weiteren, im Einzelfall interessierten Kreisen unterbreitet."""

In [14]:
tokens = list(map(lemmatize_text, text.split(" ")))

In [15]:
a1_tokens = [x for x in tokens if x in cefr[cefr.level == "A1"].lemma.values]
a2_tokens = [x for x in tokens if x in cefr[cefr.level == "A2"].lemma.values]
b1_tokens = [x for x in tokens if x in cefr[cefr.level == "B1"].lemma.values]

not_in_cefr = [x for x in tokens if x not in cefr.lemma.values]

a1_share = len(a1_tokens) / len(tokens)
a2_share = len(a2_tokens) / len(tokens)
b1_share = len(b1_tokens) / len(tokens)

print(f"A1: {a1_share:.2%}")
print(f"A2: {a2_share:.2%}")
print(f"B1: {b1_share:.2%}")
print()

print(f"Words not in CEFR:\n{sorted(set(not_in_cefr))}")

A1: 42.03%
A2: 5.80%
B1: 17.39%

Words not in CEFR:
['akzeptanz', 'berggebiet', 'bezeichnen', 'bund', 'bundesversammlung', 'dachverbände', 'derjenige', 'einzelfall', 'erheblich', 'gemeinde', 'gesetzgebung', 'kantone', 'partei', 'phase', 'richtigkeit', 'sachlich', 'sowie', 'tragweit', 'unterbreiten', 'vernehmlassungsverfahr', 'vollzugstauglichkeit', 'vorlage', 'vorverfahren', 'wirtschaftlich']
